# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 


In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [5]:
import os
from glob import glob

PRICE_DATA = os.getenv("PRICE_DATA")
# Write your code below.
parquet_files = glob(os.path.join(PRICE_DATA, "**/*parquet"), recursive = True)
dd_parquet = dd.read_parquet(parquet_files).set_index("Ticker")


In [6]:
dd_parquet.head()

Price,Date,Adj Close,Close,High,Low,Open,Volume,Year
Ticker,,,,,,,,
A,2000-01-03 00:00:00+00:00,43.382858,51.502148,56.464592,48.193848,56.330471,4674353.0,2000
A,2000-01-04 00:00:00+00:00,40.068878,47.567955,49.266811,46.316166,48.730328,4765083.0,2000
A,2000-01-05 00:00:00+00:00,37.583393,44.617310,47.567955,43.141991,47.389126,5758642.0,2000
A,2000-01-06 00:00:00+00:00,36.152378,42.918453,44.349072,41.577251,44.080830,2534434.0,2000
A,2000-01-07 00:00:00+00:00,39.165062,46.494991,47.165592,42.203148,42.247852,2819626.0,2000


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [29]:
# Write your code below.
dd_shift = dd_parquet.groupby("Ticker", group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1=x["Close"].shift(1),
        Adj_Close_lag_2=x["Adj Close"].shift(1)))

dd_rets = dd_shift.assign(
    Returns = lambda x: x["Close"]/x["Close_lag_1"]-1)

dd_feat = dd_rets.assign(
    hi_lo_range = lambda x : x['High']- x['Low']
)

print(dd_feat)

Dask DataFrame Structure:
                                  Date Adj Close    Close     High      Low     Open   Volume   Year Close_lag_1 Adj_Close_lag_2  Returns hi_lo_range
npartitions=13078                                                                                                                                    
                   datetime64[ns, UTC]   float64  float64  float64  float64  float64  float64  int32     float64         float64  float64     float64
                                   ...       ...      ...      ...      ...      ...      ...    ...         ...             ...      ...         ...
...                                ...       ...      ...      ...      ...      ...      ...    ...         ...             ...      ...         ...
                                   ...       ...      ...      ...      ...      ...      ...    ...         ...             ...      ...         ...
                                   ...       ...      ...      ...      ..

/var/folders/gv/r771h3zn7cz5dqxf_d4wg09c0000gn/T/ipykernel_35652/872089270.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_parquet.groupby("Ticker", group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [32]:
# Write your code below.
df_feat = dd_feat.compute()
df_feat

Price,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adj_Close_lag_2,Returns,hi_lo_range
Ticker,,,,,,,,,,,,
DOV,2002-01-02 00:00:00+00:00,16.045649,24.842640,24.963299,24.420326,24.869452,1022622.0,2002,NaN,NaN,NaN,0.542973
DOV,2002-01-03 00:00:00+00:00,16.340069,25.298468,25.405722,24.835936,24.876156,989355.0,2002,24.842640,16.045649,0.018349,0.569786
DOV,2002-01-04 00:00:00+00:00,16.721077,25.888363,25.955397,25.405722,25.405722,1319488.0,2002,25.298468,16.340069,0.023317,0.549675
DOV,2002-01-07 00:00:00+00:00,16.608509,25.714075,26.136387,25.687262,25.975506,1060812.0,2002,25.888363,16.721077,-0.006732,0.449125
DOV,2002-01-08 00:00:00+00:00,16.236158,25.137587,25.807922,24.869452,25.673855,976377.0,2002,25.714075,16.608509,-0.022419,0.938471
...,...,...,...,...,...,...,...,...,...,...,...,...
CTLT,2004-12-27 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,2004,NaN,NaN,NaN,NaN
CTLT,2004-12-28 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,2004,NaN,NaN,NaN,NaN
CTLT,2004-12-29 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,2004,NaN,NaN,NaN,NaN


In [35]:
df_feat = df_feat.assign(
    Return_moving_mean = lambda x : x['Returns'].rolling(10).mean())

df_feat.head(20)

Price,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adj_Close_lag_2,Returns,hi_lo_range,Return_moving_mean
Ticker,,,,,,,,,,,,,
DOV,2002-01-02 00:00:00+00:00,16.045649,24.842640,24.963299,24.420326,24.869452,1022622.0,2002,NaN,NaN,NaN,0.542973,NaN
DOV,2002-01-03 00:00:00+00:00,16.340069,25.298468,25.405722,24.835936,24.876156,989355.0,2002,24.842640,16.045649,0.018349,0.569786,NaN
DOV,2002-01-04 00:00:00+00:00,16.721077,25.888363,25.955397,25.405722,25.405722,1319488.0,2002,25.298468,16.340069,0.023317,0.549675,NaN
DOV,2002-01-07 00:00:00+00:00,16.608509,25.714075,26.136387,25.687262,25.975506,1060812.0,2002,25.888363,16.721077,-0.006732,0.449125,NaN
DOV,2002-01-08 00:00:00+00:00,16.236158,25.137587,25.807922,24.869452,25.673855,976377.0,2002,25.714075,16.608509,-0.022419,0.938471,NaN
DOV,2002-01-09 00:00:00+00:00,16.279451,25.204620,25.673855,25.144291,25.204620,745746.0,2002,25.137587,16.236158,0.002667,0.529564,NaN
DOV,2002-01-10 00:00:00+00:00,15.963385,24.715275,25.204620,24.400217,25.204620,678317.0,2002,25.204620,16.279451,-0.019415,0.804403,NaN
DOV,2002-01-11 00:00:00+00:00,15.859478,24.554394,24.956596,24.534285,24.768902,544056.0,2002,24.715275,15.963385,-0.006509,0.422312,NaN
DOV,2002-01-14 00:00:00+00:00,15.595375,24.145491,24.675055,24.138786,24.601318,493335.0,2002,24.554394,15.859478,-0.016653,0.536268,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.